# T019 · The *One-Hot Encoding* concept

Developed in the CADD seminar 2020, Volkamer Lab, Charité/FU Berlin 

Authors :

- Sakshi Misra, CADD seminar 2020, Charité/FU Berlin
- Talia B. Kimber, [Volkamer lab](https://volkamerlab.org), Charité
- Andrea Volkamer, [Volkamer lab](https://volkamerlab.org), Charité

## Aim of this talktorial

The aim of the talktorial is to perform one-hot encoding of SMILES structures on a subset of the ChEMBL dataset to gain a deeper understanding on the one-hot encoding concept and why it is useful as a pre-processing step in various machine learning algorithms. 

<img src="./images/logo.png" width="300" align='center'>

### Contents in *Theory*

- Introduction to SMILES and some of its specification rules
- Understanding categorical data and its conversion to numeric data for usage in machine learning models 
- Explanation of one-hot encoding concept and its advantages and disadvantages
- Integer encoding and the differences between integer encoding and one-hot encoding

### Contents in *Practical*

- Import necessary packages and visualize the dataframe
- Apply one-hot encoding using own implementation
  - Visualization of one-hot encoded matrix (unequal dimension)
- Function defined to pre-process the data and then apply one-hot encoding using own implementation on preprocessed data
  - Visualization of one-hot encoded matrix (equal dimension)
- Apply one-hot encoding using implementation in [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)
   - Without padding and visualization of the matrix (unequal dimension)
   - Padding after one-hot encoding is performed and visualization of the matrix (equal dimension)
   - Padding before one-hot encoding performed and visualization of the matrix (equal dimension)
- Apply one-hot encoding using implementation in [keras](https://keras.io/)
  - Without padding and visualization of the matrix (unequal dimension)
  - With padding and visualization of the matrix (equal dimension)

## References

- Theoretical background:
     - ChEMBL database: Bento, A. Patrícia, et al. "The ChEMBL bioactivity database: an update." _Nucleic acids research_ 42.D1 (2014): D1083-D1090. https://doi.org/10.1093/nar/gkt1031.
     - Blogpost: Jason Brownlee, _How to One Hot Encode Sequence Data in Python_, Machine Learning Mastery, Available from https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/, accessed November 9th, 2020.
     

- Packages used:
     - [**rdkit**](https://www.rdkit.org/docs/GettingStartedInPython.html): Greg Landrum,  _RDKit Documentation_, [PDF](https://buildmedia.readthedocs.org/media/pdf/rdkit/latest/rdkit.pdf), Release on 2019.09.1.
     - [**scikit-learn**](https://scikit-learn.org/stable/):
         - [Scikit-learn: Machine Learning in Python](https://jmlr.csail.mit.edu/papers/v12/pedregosa11a.html), Pedregosa _et al._, JMLR 12, pp. 2825-2830, 2011.
         - Jiangang Hao, et al. "A Review of Scikit-learn Package in Python Programming Language." _Journal of Education and Behavioral Statistics_ Volume: 44 issue: 3 (2019), page(s): 348-361. https://doi.org/10.3102/1076998619832248.
     - [**keras**](https://keras.io/): Book chapter: "An Introduction to Deep Learning and Keras" in *Learn Keras for Deep Neural Networks* (2019), page(s):1-16, https://doi.org/10.1007/978-1-4842-4240-7.
     - [**Matplotlib**](https://matplotlib.org/)
     - [**timeit**](https://docs.python.org/3/library/timeit.html)
     

- `SMILES encoder` function: Blogpost by iwatobipen, *encode and decode SMILES strings* , Wordpress, Available from https://iwatobipen.wordpress.com/2017/01/22/encode-and-decode-smiles-strings/, accessed November 9th, 2020.

## Theory

### ChEMBL database

- [ChEMBL](https://www.ebi.ac.uk/chembl/) is an open large-scale bioactivity database, containing molecules with drug-like properties. 
- Recent release 17 contains information extracted from  more than 51,000 publications, together with bioactivity   data sets from 18 other sources (depositors and databases). In total, there are now more than 1.3 million distinct compound structures and 12 million bioactivity data points.
- It is maintained by [European Bioinformatics Institute](https://en.wikipedia.org/wiki/European_Bioinformatics_Institute).


### SMILES structures
- [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) (Simplified Molecular Input Line Entry System) notation is a chemical notation that allows a user to represent a chemical structure of a molecule in a linear way that can be read by the computer (see "Modern Aspects of the Smiles Rearrangement" (2017), [_Chemistry A European Journal_, **Volume23, Issue38**, 8992-9008](https://doi.org/10.1002/chem.201700353) for further information.)
- It contains a chain of letters, numbers and characters that specify the atoms, their connectivity, their bond order and chirality.

#### Some SMILES specification rules
- **Atoms** - are represented by their atomic symbols. Also metal atoms are represented with symbols in square bracket, e.g. Gold `[Au]`.
- **Bonds** - single, double and triple bonds are represented by symbols `-`, `=` and `#` respectively. Aromatic bonds are represented by `*`. Single bonds are the default and therefore need not be entered. Aromatic C, O, S and N atoms are shown in lower case like 'c', 'o', 's' and 'n' or by symbol `:`, whereas aliphatic C, O, S and N atoms are shown using upper case. For example, 'CC' would mean that there is a non-aromatic carbon attached to another non-aromatic carbon by a single bond, and the computer would identify the structure as the chemical ethane `CH3CH3`.
- **Rings** - SMILES allows a user to identify ring structures by using numbers to identify the opening and closing ring atom. For example, in `C1CCCCC1`, the first carbon has a number '1' which connects by a single bond with the last carbon which also has a number '1'. The resulting structure is cyclohexane.
- **Branches** - are specified by enclosing them in parentheses, and can be nested or arranged. For example, 2-Propanol is represented by `CC(O)C`.

### What is Categorical Data?
Categorical data are variables that contain label values rather than numeric values.
Some examples include:

- A “pet” variable with the values: “dog” and “cat“.
- A “color” variable with the values: “red“, “green” and “blue“.
- A “place” variable with the values: “first”, “second” and “third“.

In terms of bioinformatics, if we are using a machine learning classifier to classify cancerous and normal tissues cells, we can have label values say "Lung Cancer", "Breast Cancer", "Liver Cancer" and "Healthy Controls".
We first need to one-hot encode these categorical label values and then we can apply binary or multi-class classifiers to achieve classification results.


#### What is the problem with categorical data?
Machine Learning is, after all, a bunch of mathematical operations translated to a computer via low-level programming languages. Computers are brilliant when dealing with numbers. So we must somehow convert our input data to numbers. 
There are many machine learning algorithms which cannot operate on categorical data directly so they must be converted to a numerical form so all our input variables and output variables will be numeric (see [**Blogpost**](https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/): Alakh Sethi, _One-Hot Encoding vs. Label Encoding using Scikit-Learn_, Analytics Vidya, accessed March 6th, 2020 for further information).

<div style="text-align: center;">
<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/03/external-content.duckduckgo.com_.jpeg" alt="Drawing" style="max-width: 500px; width:400%;"/>
</div>

**Figure 1** displays the categorical encoding required for computers to understand the input. The figure comes from this [blogpost](https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/).

#### How to convert categorical data to numerical data?
There are many ways to convert categorical values into numerical values. Each approach has its own positive and negative impact on the feature set. Hereby, two main methods will be the focus: `One-Hot` encoding and `Label` encoding.
Both of these encoders are part of the scikit-learn library (one of the most widely used Python libraries) and are used to convert text or categorical data into numerical data which the model expects and can perform with.

### The *One-Hot Encoding* concept
The one-hot encoding is a vector representation where all the elements of the vector are set to `0` except one, which has `1` as its value. For example, `[0 0 0 1 0 0]` is a one-hot vector.
Simply put, one-hot encoding, also known as binary encoding, is a binary representation of categorical variables as binary vectors. 

The figures shown below help us gain an overall idea of the one-hot encoding concept, see [<i>BMC Bioinformatics.</i> (2018), <b>19</b>,526](https://doi.org/10.1186/s12859-018-2523-5) for further information.


<div style="text-align: center;">
    <img src="https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs12859-018-2523-5/MediaObjects/12859_2018_2523_Fig1_HTML.png?as=webp" style="max-width: 500px; width:400%;" />
</div>



**Figure 2a** shows the one-hot encoding of four DNA nucleotides, a filter kernel with one-dimensional convolutional neural network.

**Figure 2b** shows the one-hot encoding of toluene and applying a one-dimensional convolutional neural network to the SMILES linear representation of the chemical compound toluene.
 

Let us take a deeper look into the concept with the help of a simple example that will describe the basic concept of one-hot encoding, why it is useful and how one can approach it.

#### Why use one-hot encoding?
One-hot encoding allows the representation of categorical data to be more expressive.
It is difficult for many machine learning algorithms to work with categorical data directly that's why the label values which are categorical must be converted into numbers first as a preprocessing step. This is required for both input and output variables that are categorical.
We could use an integer encoding directly. This may work for problems where there is a natural ordinal relationship between the categories, and in turn the integer values, such as labels for temperature ‘cold’, warm’, and ‘hot’.
There may be problems when there is no ordinal relationship and allowing the representation to lean on any such relationship might be damaging to learning to solve the problem. An example might be the labels ‘dog’ and ‘cat’.


#### Example of one-hot encoding
Let us take a look at a very simple example to understand this concept. Assume we have the “color” variable which has three labels `RED` , `BLUE` and `GREEN`.
All these labels must be converted into numeric form in order to work with our machine learning algorithm. This can be done by creating three new columns having all three labels and use `1` for the color and `0` for the other colors as shown in Figure 3 (see article: "*Building a One Hot Encoding Layer with TensorFlow*", George Novack, [towardsdatascience](https://towardsdatascience.com/building-a-one-hot-encoding-layer-with-tensorflow-f907d686bf39) for more details).


<div style="text-align: center;">
    <img src="images/OneHotEncoding_eg.png" style="max-width: 700px; width:150%;" />
</div>

**Figure 3** shows the visual demonstration of one-hot encoding done on the variable "color". 

####  Advantages of one-hot encoding 
-  If the cardinality (the number of categories) of the categorical features is low (relative to the amount of data), one-hot encoding will work best. We can use it as input to any machine learning model.
-  We can create binary representations of label values which can be useful for binary classification.


####  Disadvantages of one-hot encoding 
-  Increase in dimensionality, after adding several columns based on categorical variables. The dataset will have more dimensions than before and in result may increase the computational cost.
- There is a high chance of multicollinearity due to dummy variables which can affect the performance of our model.
-  Increase [sparsity](https://en.wikipedia.org/wiki/Sparse_matrix) (a sparse array is a matrix in which most of the elements are zero, one-hot encoding can result in increasing the sparsity of the dataset).

### Other similar transformation: Integer encoding (label encoder)

[Label Encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html), or integer encoding, is a popular encoding technique for handling categorical variables and is easily reversible. In this technique, each label is assigned a unique integer based on alphabetical ordering, so that machines can work with it properly.
Machine learning algorithms can then decide in a better way on how labels must be operated on.
It is an important preprocessing step for the structured dataset in supervised learning.

#### Example of integer encoding
Let us take a similar example as above: we have a color variable and we can assign `red` as `0`, `green` as `1`, and `blue` as `2` as shown in Figure 4 (see article: "*Know about Categorical Encoding, even New Ones!*", Ahmed Othmen, [towardsdatascience](https://towardsdatascience.com/know-about-categorical-encoding-even-new-ones-c266227b9cbd) for more details).



![OneHotEncoding Example](images/label_encoding_example.png)

**Figure 4** shows the visual demonstration of label encoding of color variable.

### Difference between label and one-hot encoding

There is not much difference between these two encoding techniques, it mainly depends on the type of data and model we are using. For example if we have categorical features which are not ordinal (dog or cat) then we can use one-hot encoding. Label encoding works best with ordinal data like `Good=0, Better=1, Best=2`.
Also when there are more categorical variables then its good to choose label encoding just to avoid high memory consumption and sparsity.

### Further readings

This section lists some resources for further reading:

- [What is one-hot encoding and when is it used in data science?](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)
- [Categorical encoding using Label-Encoding and One-Hot-Encoder](https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd#:~:text=One%2DHot%20Encoding%20in%20Python&text=OneHotEncoder%20from%20SciKit%20library%20only,apply%20OneHotEncoder%20on%20column%20Bridge_Types_Cat.)
- Hirohara, M., Saito, Y., Koda, Y. et al. Convolutional neural network based on SMILES representation of compounds for detecting chemical motif. _BMC Bioinformatics_ **19**, 526 (2018). https://doi.org/10.1186/s12859-018-2523-5.
- [How one can use matplotlib.pyplot.imshow() in Python](https://www.geeksforgeeks.org/matplotlib-pyplot-imshow-in-python/)

## Practical

### Imports

In [ ]:
# Ignore warnings for readibility
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import statements
import pandas as pd
import numpy as np
from pathlib import Path
from rdkit.Chem import Draw, PandasTools
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from timeit import default_timer as timer

### Load and draw molecules

Using the `Pandas` library, we first load the data and then draw the molecules using the `rdkit.draw` function. Then we apply different implementations of `one-hot encoding` to the SMILES structures.

Let's load the data and quickly analyze its column values and check if there are any missing values:

In [ ]:
# Set path to this notebook
HERE = Path(_dh[-1])
DATA = HERE / "data"

df = pd.read_csv('DATA/CHEMBL25_activities_EGFR.csv',
                 lineterminator='\n', index_col=0)  # NBVAL_CHECK_OUTPUT

In [ ]:
# Check the dimension and missing value of the data
print("Shape of dataframe : ", df.shape)
df.info()

# Look at head
df.head(3)  # NBVAL_CHECK_OUTPUT

In [ ]:
# Using pandas tools and the Draw method,
# we can visualize the molecules with their ChEMBL ID
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='canonical_smiles')
Draw.MolsToGridImage(list(df.ROMol[0:10]),
                     legends=list(df.chembl_id[0:20]),
                     molsPerRow=5)  # NBVAL_CHECK_OUTPUT

In [ ]:
# Renaming column name ROMol to 2D_Figure to get the proper idea of the column
df = df.rename(columns={'ROMol': '2D_Figures'})
df.head(2)  # NBVAL_CHECK_OUTPUT

### Apply one-hot encoding using own implementation

We now define our own function which will be useful to create the one-hot encoded matrix of the SMILES strings.

In [ ]:
# Create dataset of all possible SMILES characters
SMILES_CHARS = [' ', '#', '%', '(',
                ')', '+', '-', '.', '/', '0', '1', '2', '3',
                '4', '5', '6', '7', '8', '9',
                '=', '@', 'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M',
                'N', 'O', 'P', 'R', 'S', 'T', 'V', 'X', 'Z',
                '[', '\\', ']', 'a', 'b', 'c', 'e', 'g', 'i',
                'l', 'n', 'o', 'p', 'r', 's', 't', 'u']

In [ ]:
# Convert the dataset into a dictionary
smi2index = dict((char, index) for index, char in enumerate(SMILES_CHARS))

In [ ]:
# Function defined to create one-hot encoded matrix
def smiles_encoder(smiles):
    """
    One-hot encodes SMILES strings
    using all possible 56 pre-defined characters.

    Parameters
    ----------
    smiles : str
          SMILES string of a compound.

    Returns
    -------
    smiles_matrix: ndarray
          One-hot encoded matrix of shape
          (defined SMILES_CHARS, length of individual SMILES).
    """
    smiles_length = len(smiles)
    smiles_matrix = np.zeros((len(SMILES_CHARS), smiles_length), dtype=int)
    for index, char in enumerate(smiles):
        smiles_matrix[smi2index[char], index] = 1
    return smiles_matrix

In [ ]:
# Apply the function over the canonical SMILES strings
start = timer()
df['own_OHE'] = df['canonical_smiles'].apply(smiles_encoder)
end = timer()
df.head(3)  # NBVAL_CHECK_OUTPUT

In [ ]:
# Print time required for execution
smiles_encoder_time = end - start
print(f"Time to execute the function: "
      f"{smiles_encoder_time:.2f} secs")  # NBVAL_CHECK_OUTPUT

#### Visualization with Matplotlib.imshow() function
`Matplotlib` is a plotting library for the python programming language and `Pyplot` is a state-based interface to a matplotlib module which provides a MATLAB-like interface.
The [imshow()](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html) function in the pyplot module of the matplotlib library is used to display data as an image i.e. on a 2D space.

We now visualize our one-hot encoded matrix using `imshow()` by defining our own `ohe_plot` function as shown below.

In [ ]:
def ohe_plot(ohe_matrix, smiles_char):
    """
    Visualize one-hot encoded matrix
    using matplotlib imshow() function.

    Parameters
    ----------
    ohe_matrix : ndarray
       One-hot encoded matrix of shape
       (defined SMILES_CHARS, length individual SMILES).
    smiles_char : dict
        Dictonary with all possible SMILES characters.

    Returns
    ------
    None

    """
    im = plt.imshow(ohe_matrix, cmap='hot', interpolation='None')
    plt.colorbar(im, orientation='horizontal')
    plt.xlabel('Length of SMILES string')
    plt.ylabel(f'Char in SMILES ({len(smiles_char)})')
    plt.title('Visualization of one-hot encoded matrix')
    plt.show()
    print('Shape of one-hot matrix : ', ohe_matrix.shape)
    return None

In [ ]:
# choose index
index = 0
ohe_plot(df.iloc[index]['own_OHE'], SMILES_CHARS)  # NBVAL_CHECK_OUTPUT
print('Associated canonical SMILES: ', df.iloc[index]['canonical_smiles'])

In [ ]:
index = 100
ohe_plot(df.iloc[index]['own_OHE'], SMILES_CHARS)  # NBVAL_CHECK_OUTPUT
print('Associated canonical SMILES: ', df.iloc[index]['canonical_smiles'])

- First, as we notice from the visualizations above, SMILES strings could have unequal dimension since their string length might differ. For machine learning application, having equal dimension throughout the data set is required. In order to achieve this, we first search for the string with the maximum length using the [len()](https://www.geeksforgeeks.org/python-string-length-len/) method. By implementing the above method, we can get the maximum length of the SMILES strings and pass it as an argument in our function for all the strings.

- Second, in the above function, we have created a dataset of 56 possible SMILES characters, but to optimize our one-hot encoding, we reconsidered using all the 56 characters. We can assume that all the characters won't be present in our SMILES structures, so we look for all the unique characters present in the SMILES data set.

- Third, many elements in the periodic table have two characters in their name. For example 'Cl' stands for chloride. This double-character phenomenon is commonly present in SMILES but if we use the above `smiles_encoder` function then we will be splitting 'Cl' into two characters, namely 'C' and 'l', and that would lead to discrepancies, so searching for each unique character and encoding them may not be optimal. Hence, we search for all the two alphabetic element in our SMILES by comparing the atoms present in our strings with all the possible elements present in the periodic table and replaced all the two alphabetic elements with one character, for example 'Cl' is changed to 'L'.

### Function defined to pre-process the SMILES data

In [ ]:
# Create function used to preprocessed the data
def preprocessing_data(df):
    """
    Preprocess the SMILES structures in a data set.

    Parameters
    ----------
    df : pandas.DataFrame
       Dataframe which requires preprocessing.

    Returns
    -------
    df : pandas.DataFrame
        Dataframe with new processed canonical SMILES column.
    unique_char : list
        List with unique characters present in SMILES.
    SMILES_maxlen : int
        Maximum length of SMILES string.
    """

    # Calculate max length of the SMILES strings
    SMILES_maxlen = df["canonical_smiles"].str.len().max()

    # Search for unique characters in our SMILES strings
    unique_char = set(df.canonical_smiles.apply(list).sum())
    upper_chars = ['C', 'O', 'F', 'P', 'N', 'S', 'H', 'B', 'I']
    lower_chars = ['l', 'o', 'r', 'n', 'e', 'c', 's']

    # List of all possible periodic elements
    periodic_elements = ['Ac',
                         'Al', 'Am', 'Sb', 'Ar', 'As', 'At', 'Ba',
                         'Bk', 'Be', 'Bi', 'Bh', 'B', 'Br', 'Cd', 'Ca',
                         'Cf', 'C', 'Ce', 'Cs', 'Cl', 'Cr', 'Co', 'Cn',
                         'Cu', 'Cm', 'Ds', 'Db', 'Dy', 'Es', 'Er', 'Eu',
                         'Fm', 'Fl', 'F', 'Fr', 'Gd', 'Ga', 'Ge', 'Au',
                         'Hf', 'Hs', 'He', 'Ho', 'H', 'In', 'I', 'Ir',
                         'Fe', 'Kr', 'La', 'Lr', 'Pb', 'Li', 'Lv',
                         'Lu', 'Mg', 'Mn', 'Mt', 'Md', 'Hg', 'Mo',
                         'Mc', 'Nd', 'Ne', 'Np', 'Ni', 'Nh', 'Nb',
                         'N', 'No', 'Og', 'Os', 'O', 'Pd', 'P',
                         'Pt', 'Pu', 'Po', 'K', 'Pr', 'Pm', 'Pa',
                         'Ra', 'Rn', 'Re', 'Rh', 'Rg', 'Rb', 'Ru', 'Rf',
                         'Sm', 'Sc', 'Sg', 'Se', 'Si', 'Ag', 'Na',
                         'Sr', 'S', 'Ta', 'Tc', 'Te', 'Ts', 'Tb', 'Tl', 'Th',
                         'Tm', 'Sn',
                         'Ti', 'W', 'U', 'V', 'Xe', 'Yb', 'Y', 'Zn', 'Zr']

    # 'two_charc_elements' is a list that contains two letter elements
    # which are valid when compared with all the possible periodic elements.
    two_charc_elements = []
    for upper in upper_chars:
        for lower in lower_chars:
            ch = upper + lower
            if (ch in periodic_elements):
                two_charc_elements.append(ch)

    # 'two_charc_elements_smiles' is a set that contains all the possible
    # two letter elements in our SMILES strings, that is specific to our dataset.
    two_charc_elements_smiles = set()
    for char in two_charc_elements:
        for index in range(len(df)):
            if (df['canonical_smiles'].iloc[index].find(char) != -1):
                two_charc_elements_smiles.add(char)
    # Create a new column having processed canonical SMILES
    df['processed_canonical_smiles'] = ""

    # Replaced all the two letter elements found
    # (Cl, Br, Cn, Se, @@) with one character
    for index in range(len(df)):
        element = df['canonical_smiles'].iloc[index]
        element = element.replace("Cl", "L")
        element = element.replace("Br", "R")
        element = element.replace("Cn", "X")
        element = element.replace("Se", "Z")
        element = element.replace("@@", "$")
        df['processed_canonical_smiles'].iloc[index] = element
    unique_char = set(df.processed_canonical_smiles.apply(list).sum())
    return unique_char, df, SMILES_maxlen

**Note:** This function is highly specific to the above dataset used, because we have replaced two letter elements that are particularly found in the used dataset. One can add more two letter elements present in their specific SMILES.

### Apply one-hot encoding using own implementation on preprocessed data

Now we have used the same function `smiles_encoder` but on processed canonical strings.


In [ ]:
# Calling function
unique_char, df, SMILES_maxlen = preprocessing_data(df)

In [ ]:
# Create dictionary of the unique char dataset
smi2index = dict((char, index) for index, char in enumerate(unique_char))

In [ ]:
# Function defined to create one-hot encoded matrix
def smiles_encoder(smiles, maxlen):
    """
    Function defined using all unique characters in our
    processed canonical smiles structures created
    from preprocessed function.

    Parameters
    ----------
    smiles : str
         SMILES data in string format.
    maxlen : int
         Maximum length of the SMILES string.

    Returns
    -------
    smiles_matrix : ndarray
         One-hot encoded matrix of fixed shape
         (unique char in smiles, max smile length).
    """
    smiles_matrix = np.zeros((len(unique_char), maxlen))
    for index, char in enumerate(smiles):
        smiles_matrix[smi2index[char], index] = 1
    return smiles_matrix

In [ ]:
# Apply the function over the SMILES strings
start = timer()
df['Unique_char_OHE'] = df['processed_canonical_smiles'].apply(
                        smiles_encoder, maxlen=SMILES_maxlen)
end = timer()
df.head(3)  # NBVAL_CHECK_OUTPUT

In [ ]:
# Print time required for execution
smiles_encoder_equal_dim = end - start
print(f"Time to execute the function: "
      f"{smiles_encoder_equal_dim:.2f} secs")  # NBVAL_CHECK_OUTPUT

#### Visualization with Matplotlib.imshow() function

In [ ]:
# choose index
index = 0
ohe_plot(df.iloc[index]['Unique_char_OHE'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

In [ ]:
index = 100
ohe_plot(df.iloc[index]['Unique_char_OHE'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

From the above visualizations, we can conclude that the dimensions are equal for all the strings' one-hot encoded matrix using `preprocessing_data` and `smiles_encoder` functions.

### Apply one-hot encoding using implementation in Scikit-learn
Now, we proceed with our second implementation of one-hot encoding from scikit-learn. We can use [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) from the `sklearn` library but it only takes numerical categorical values, hence any value of string type should be [label_encoded](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) first before one-hot encoded.

In [ ]:
# Split the SMILES into list of characters
def split(string_data):
    """
    Split the SMILES strings into characters array.

    Parameters
    ----------
    string_data : str
        SMILES data in string format.

    Returns
    -------
    list
        List of characters in the string data.
    """
    return list(string_data)

In [ ]:
# Use Scikit-learn implementation of one-hot encoding
def sklearn_OHE(smiles, islaterpadding):
    """
    Label and one-hot encodes the SMILES
    using sklearn LabelEncoder and OneHotEncoder implementation.
    
    Parameters
    ----------
    smiles : str
        SMILES string of a compound.
    islaterpadding : bool
        paramater is `True` if `later_padding` is required,
        `False` otherwise.
          
    Returns
    -------
    onehot_encoded : ndarray
        One-hot encoded matrix of shape
        (chars in individual SMILES, length of individual SMILES).
    """
    # Integer encoding
    canonical_char = split(smiles)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(canonical_char)
    # one-hot encoding
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    if (islaterpadding == True):
        onehot_encoded = later_padding(onehot_encoded)
    onehot_encoded = onehot_encoded.transpose()
    return onehot_encoded

Running the functions above first give integer encoded SMILES of the labels and finally one-hot encode the SMILES structures.

By default, the OneHotEncoder class will return a more efficient sparse encoding which can be useful in some applications but in this case, we disabled the sparse return type by setting the `sparse=False` argument.

#### Without padding (unequal dimension)

We can use the `sklearn_OHE` function defined above to create the one-hot encoded matrix but this will again create unequal dimensions of the matrix because it will first label encode all the characters present in the SMILES strings (individually) and then one-hot encode them.

In [ ]:
# Apply the function over the processed canonical SMILES strings
start = timer()
df['sklearn_OHE_no_padding'] = df['processed_canonical_smiles'].apply(
                                 sklearn_OHE, islaterpadding=False)
end = timer()
df.head(2)

In [ ]:
# Print time required for execution
sklearn_time_without_padding = end - start
print(f"Time to execute the function: "
      f"{sklearn_time_without_padding:.2f} secs")  # NBVAL_CHECK_OUTPUT

##### Visualization with Matplotlib.imshow() function

In [ ]:
# choose index
index = 0
ohe_plot(df.iloc[index]['sklearn_OHE_no_padding'],
         df.iloc[index]['sklearn_OHE_no_padding'])  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

In [ ]:
index = 100
ohe_plot(df.iloc[index]['sklearn_OHE_no_padding'],
         df.iloc[index]['sklearn_OHE_no_padding'])  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

It can be confirmed from above visualization that the one-hot encoded matrix from the sklearn implementation does not have equal dimensions because sklearn's `OneHotEncoder` function takes individual strings as input.

So we equalize dimensions of the one-hot encoded matrix by adding **padding** which simply means adding zeros to it. It can be either done after one-hot encoding is performed on the SMILES strings or before, meaning after we label encode the SMILES characters.

We discuss both scenarios in next sections.

#### With padding (equal dimension)

##### Padding after one-hot encoding is perfomed

In [ ]:
# Defined function to add padding after one-hot encoding
def later_padding(ohe_matrix):
    """
    Add horizontal and vertical padding
    to the given matrix using numpy.pad() function.

    Parameters
    ----------
    ohe_matrix: ndarray
            Character array.
            
    Returns
    -------
    padded_matrix: ndarray
           Padded one-hot encoded matrix of
           shape (unique char in smiles, max smile_length).
   """
    index_padding = np.pad(ohe_matrix, ((0, SMILES_maxlen-len(ohe_matrix)),
                                        (0, 0)), 'constant')
    column_padding = np.pad(ohe_matrix, ((0, SMILES_maxlen),
                                         (0, len(unique_char)-len(
                                          ohe_matrix[0]))), 'constant')
    ohe_matrix = np.append(ohe_matrix, index_padding, axis=0)
    ohe_matrix = np.append(ohe_matrix, column_padding, axis=1)
    return ohe_matrix

In [ ]:
# Apply the function over the processed canonical smiles strings
start = timer()
df['sklearn_OHE_later_padding'] = df['processed_canonical_smiles'].apply(
                                     sklearn_OHE, islaterpadding=True)
end = timer()
df.head(2)

In [ ]:
# Print time required for execution
sklearn_time_later = end - start
print(f"Time to execute the function: "
      f"{sklearn_time_later:.2f} secs")  # NBVAL_CHECK_OUTPUT

Visualization with Matplotlib.imshow() function.

In [ ]:
# TODO: FIX
# y label is not 37
# matrices don't have equal dimensions.

In [ ]:
# choose index
index = 0
ohe_plot(df.iloc[index]['sklearn_OHE_later_padding'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

In [ ]:
index = 100
ohe_plot(df.iloc[index]['sklearn_OHE_later_padding'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

Now we can observe that the dimensions are equal for all the SMILES strings.

##### Padding before one-hot encoding is perfomed

In this case we have added padding just after the label encoding or before the one-hot encoding is performed on SMILES.

In [ ]:
def initial_padding(canonical_char, max_len):
    """
    Add padding to the given list
    before one-hot encoding is performed.

    Parameters
    ----------
    canonical_char : array
       Character array.
    max_len : int
       Maximum length of SMILES string.

    Returns
    -------
    padded_list : list
       Padded character list of SMILES string.
    """
    padded_list = np.pad(canonical_char, (0,
                         max_len-len(canonical_char)), 'constant')

    return padded_list

In [ ]:
def initial_padding_and_encoding(smiles):
    """
    One-hot encodes initially padded SMILES strings
    using sklearn_OneHotEncode function.

    Parameters
    ----------
    smiles : str
       SMILES string.

    Returns
    -------
    ndarray
      One-hot encoded matrix with intial padding of shape
      (unique char in smiles, max smile_length).
    """
    # call 'split' function on smiles
    canonical_char = split(smiles)
    # call 'initial_padding' function on splitted list of characters
    canonical_char_padded = initial_padding(canonical_char, SMILES_maxlen)
    # call 'sklearn_OHE' function
    sklearnOHC = sklearn_OHE(canonical_char_padded, False)
    sklearnOHC = sklearnOHC.transpose()
    column_padding = np.ndarray(shape=(SMILES_maxlen,
                                len(unique_char)-len(sklearnOHC[0])))
    column_padding.fill(0)
    sklearnOHC = np.append(sklearnOHC, column_padding, axis=1)
    return sklearnOHC.transpose()

In [ ]:
# Apply the function over the processed canonical SMILES strings
start = timer()
df['sklearn_OHE_initial_padding'] = df['processed_canonical_smiles'].apply(
                                      initial_padding_and_encoding)
end = timer()
df.head(2)

In [ ]:
# Print time required for execution
sklearn_time_initial = end - start
print(f"Time to execute the function: "
      f"{sklearn_time_initial:.2f} secs")  # NBVAL_CHECK_OUTPUT

Visualization with Matplotlib.imshow() function 

In [ ]:
ohe_plot(df.iloc[0]['sklearn_OHE_initial_padding'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[0]['processed_canonical_smiles'])

In [ ]:
ohe_plot(df.iloc[100]['sklearn_OHE_initial_padding'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[100]['processed_canonical_smiles'])

### Apply one-hot encoding using implementation in Keras

Keras is also a very powerful and highly used library, especially employed in deep learning tasks. 
There may be a case where we have sequences or strings that are already integer encoded, then in that case we can use the function called [to_categorical()](https://keras.io/api/utils/) provide by the keras library to one-hot encode integer data directly, but it always should be integer which may not have a real ordinal relationship and are really just placeholders for labels.

In [ ]:
# Use keras implementation of one-hot encoding
def keras_OHE(smiles, islaterpadding):
    """
    One-hot encodes the SMILES using keras
    implementation.
    
    Parameters
    ----------
    canonical_char : array
        Canonical character array.
    islaterpadding : bool
         The paramater is `True` if later_padding is required,
         `False` otherwise.
        
    Returns
    -------
    encoded : ndarray
        One-hot encoded matrix of shape
        (chars in SMILES, length of SMILES).
    """
    # apply 'split' function on smiles string
    canonical_char = split(smiles)
    # integer encoding
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(canonical_char)
    # one-hot encoding
    encoded = to_categorical(integer_encoded)
    if (islaterpadding == True):
        encoded = later_padding(encoded)
    encoded = encoded.transpose()
    return encoded

#### Without padding (unequal dimension)

In [ ]:
# Apply the function over the processed canonical SMILES strings
start = timer()
df['keras_OHE_without_padding'] = df['processed_canonical_smiles'].apply(
                            keras_OHE, islaterpadding=False)
end = timer()
df.head(2)

In [ ]:
# Print time required for execution
keras_time_no_padding = end - start
print(f"Time to execute the function: "
      f"{keras_time_no_padding:.2f} secs")  # NBVAL_CHECK_OUTPUT

Visualization with Matplotlib.imshow() function 

In [ ]:
ohe_plot(df.iloc[0]['keras_OHE_without_padding'],
         df.iloc[0]['keras_OHE_without_padding'])  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[0]['processed_canonical_smiles'])

In [ ]:
ohe_plot(df.iloc[100]['keras_OHE_without_padding'],
         df.iloc[100]['keras_OHE_without_padding'])  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[100]['processed_canonical_smiles'])

#### With padding (equal dimension)

In [ ]:
# Apply the function over the Processed_canonical_smiles strings
start = timer()
df['keras_OHE_padding'] = df['processed_canonical_smiles'].apply(
                                    keras_OHE, islaterpadding=True)
end = timer()
df.head(2)

In [ ]:
# Print time required for execution
keras_time_padding = end - start
print(f"Time to execute the function: "
      f"{keras_time_padding:.2f} secs")  # NBVAL_CHECK_OUTPUT

Visualization with Matplotlib.imshow() function

In [ ]:
# TODO: FIX
# y label is not 37
# matrices don't have equal dimensions.

In [ ]:
# choose index
index = 0
ohe_plot(df.iloc[index]['keras_OHE_padding'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

In [ ]:
index = 100
ohe_plot(df.iloc[index]['keras_OHE_padding'],
         unique_char)  # NBVAL_CHECK_OUTPUT
print('Associated processed canonical SMILES: ',
      df.iloc[index]['processed_canonical_smiles'])

In [ ]:
# Final dataframe with all the one-hot encoded matrices
df.head(3)  # NBVAL_CHECK_OUTPUT

## Discussion 

As we notice from the simulations above, the execution time varies with different implementations:

- **Unequal dimension** (when no padding was performed)
    - Unexpectedly, our own `smiles_encoder` function works the best with 0.89 sec execution time followed by the implementation in keras with 1.48 sec and one-hot encoding with the scikit-learn implementation executed with 2.42 secs, the highest. But since the dimensions differ, machine learning models cannot be applied.
    
- **Equal dimension** (when padding was performed)
     - Surprisingly, even after creating equal dimensions (adding padding), our own functions `smiles_encoder` along with `preprocessing_data` again outperform the other two implementations with execution time of 1.02 secs whereas scikit-learn required approximately 3.0 secs and keras 1.85 secs. One explanation for this difference may be the additional padding performed on the strings.
     
     
We also stress the time execution difference using scikit-learn when padding is performed:

- **Before one-hot encoding** : Time required is 3.0 secs approx.
- **After one-hot encoding** : Time required is 2.7 secs approx.

One possible reason for this time difference could be that if we perform padding after label encoding, we will have more characters to one hot encode as compared to just pad after one-hot encoding is accomplished.

**Note:** Execution times might slighty differ depending on the environment used.

### Challenges 

There were several challenges faced during the task, for instance:

- Making equal dimensions of the one-hot encoded matrix.

- Replacing two letter elements with unique characters.

- After searching for two letter elements, we found a specific element `Sc` which is a metallic element. In our dataset both 'S' and 'c' elements are present individually as well. So if we would have replaced `Sc` with single a letter element then it might have effected our actual strings. So we assumed that since `Sc` is a metallic element and is rarely present in SMILES, we did not replace this element.

### Quiz

- Why is it required to have equal dimensions of the one-hot encoded matrix ?
- Is there any other way to pre-process the data ?
- How and which machine learning models can be applied on the above dataset ?